## modeling of estimation on sqft price

In [1]:
import os,sys
from sqlalchemy import create_engine
import numpy as np
import pandas as pd

cwd = os.getcwd()
lib_path = os.path.abspath(os.path.join(cwd, 'lib'))
sys.path.append(lib_path)
from plot_utils import *

ServerName = "localhost"
Database = "sdra"
engine = create_engine('postgresql://postgres:@' + ServerName + ':5432/' + Database)

### dataset basics

In [9]:
# all valid transactions of valid property with addresses
sql = '''SELECT * FROM property_estimate'''
df_all = pd.read_sql_query(sql, engine)
df_all['date'] = pd.to_datetime(df_all['date'])
df_all['sold_year'], df_all['sold_month'] = df_all['date'].dt.year, df_all['date'].dt.month
print(df_all.shape)
df_all[:3]

(1449003, 16)


,pin,year_built,sqft,num_bed,num_bath,pool,view,street,city,zip,sold_price,date,sold_age,sqft_price,sold_year,sold_month
0,5754330300,1955.0,1120,3.0,15.0,N,N,HILLTOP,CHULA VISTA,91911,447000,2017-10-24,62.0,399,2017,10
1,1657014212,1993.0,1016,2.0,20.0,None,None,GENOA,OCEANSIDE,92056,397000,2017-10-19,24.0,390,2017,10
2,5741930100,1960.0,1893,3.0,20.0,Y,N,ROBERT,CHULA VISTA,91910,547000,2017-09-27,57.0,288,2017,9


### more cleaning can be done here
* eg. feature year_built are not clean yet
* filter out sqft_price >= 1000

In [10]:
df_all_valid = df_all[(df_all['sqft_price']<1000)
                    &(df_all['year_built'] > 0)
                    ]
df_all_valid.shape

(1444802, 16)

In [13]:
# check the date of last transaction
print(df_all_valid['date'].max())

2017-10-24 00:00:00


In [14]:
# get only transactions in last one year
df_last_year = df_all_valid[(df_all_valid['date'] >= '2016-10-01') & (df_all_valid['date'] < '2017-10-01')]
df_last_year.shape

(37508, 16)

### regression model 
* target: sqft_price
* features: ...
* data set: last 1 year, 2 years, ...
* try different models